In [1]:
import requests
import os
import pandas as pd
import datetime
import json

In [2]:
bearer_token = "BQCxvHrCCSu-rVS4zLv5NB4ZXb4fidtmVI6NlF1rwRV5HIzZ9r8WR5QDM5rm2pRwfCQSDRbYU2A0wBDtL20jlPD-M4AMDtrbxKH0yN9A3B6hWybJTpUR4gppVSI36qlWXjN4grhs7el7-gcw6mSfx_oq17OPJdcYoGgniTvt5fEZ275fMjErDUyv8aCwCStOJ91FqS2M7kiPtREgrASD5elJ_vutvfIno3xOs81HuEQmYx_4w0cX2xI0X8vgUuF6"

In [3]:
endpoint = 'https://charts-spotify-com-service.spotify.com/auth/v0/charts/{chart}-{country}-weekly/{date}'

In [4]:
database_dir = 'data'
os.makedirs(database_dir, exist_ok=True)

In [5]:
headers = {
    'Authorization': f'Bearer {bearer_token}',
}

In [6]:
countries = {
    "GLOBAL": "Global",
    "AR": "Argentina",
    "AU": "Australia",
    "AT": "Austria",
    "BY": "Belarus",
    "BE": "Belgium",
    "BO": "Bolivia",
    "BR": "Brazil",
    "BG": "Bulgaria",
    "CA": "Canada",
    "CL": "Chile",
    "CO": "Colombia",
    "CR": "Costa Rica",
    "CY": "Cyprus",
    "CZ": "Czech Republic",
    "DK": "Denmark",
    "DO": "Dominican Republic",
    "EC": "Ecuador",
    "EG": "Egypt",
    "SV": "El Salvador",
    "EE": "Estonia",
    "FI": "Finland",
    "FR": "France",
    "DE": "Germany",
    "GR": "Greece",
    "GT": "Guatemala",
    "HN": "Honduras",
    "HK": "Hong Kong",
    "HU": "Hungary",
    "IS": "Iceland",
    "IN": "India",
    "ID": "Indonesia",
    "IE": "Ireland",
    "IL": "Israel",
    "IT": "Italy",
    "JP": "Japan",
    "KZ": "Kazakhstan",
    "LV": "Latvia",
    "LT": "Lithuania",
    "LU": "Luxembourg",
    "MY": "Malaysia",
    "MX": "Mexico",
    "MA": "Morocco",
    "NL": "Netherlands",
    "NZ": "New Zealand",
    "NI": "Nicaragua",
    "NG": "Nigeria",
    "NO": "Norway",
    "PK": "Pakistan",
    "PA": "Panama",
    "PY": "Paraguay",
    "PE": "Peru",
    "PH": "Philippines",
    "PL": "Poland",
    "PT": "Portugal",
    "RO": "Romania",
    "SA": "Saudi Arabia",
    "SG": "Singapore",
    "SK": "Slovakia",
    "ZA": "South Africa",
    "KR": "South Korea",
    "ES": "Spain",
    "SE": "Sweden",
    "CH": "Switzerland",
    "TW": "Taiwan",
    "TH": "Thailand",
    "TR": "Turkey",
    "AE": "UAE",
    "UA": "Ukraine",
    "GB": "United Kingdom",
    "UY": "Uruguay",
    "US": "USA",
    "VE": "Venezuela",
    "VN": "Vietnam"
}

In [7]:
all_counties = list(countries.keys())

In [8]:
chartTypes = {
    "album": "album",
    "artist": "artist",
    "song": "regional",
}

In [9]:
def get_weekly_song_rankings(start_date, country_code, chartType):
    response = requests.get(endpoint.format(country=country_code, date=start_date, chart=chartType), headers=headers)
    if response.status_code == 200:
        data = response.json()
        return data, response.status_code
    else:
        return None, response.status_code

In [10]:
date_format = '%Y-%m-%d'

In [11]:
def weekly_to_lines(weekly_json):
    rankings = []
    for entry in weekly_json['entries']:
        ranking = dict(entry['chartEntryData'])
        ranking['artist_id'] = entry['trackMetadata']['trackUri'].split(':')[-1]
        ranking['country'] = weekly_json['displayChart']['chartMetadata']['dimensions']['country']
        ranking['date'] = weekly_json['displayChart']['date']
        ranking['chartType'] = weekly_json['displayChart']['chartMetadata']['dimensions']['chartType']  
        ranking['track_name'] = entry['trackMetadata']['trackName']
        ranking['artists'] = []
        for artist in entry['trackMetadata']['artists']:
            ranking['artists'].append(artist['name']) 
        ranking['labels'] = []
        for label in entry['trackMetadata']['labels']:
            ranking['labels'].append(label['name'])
        ranking['image_link'] = entry['trackMetadata']['displayImageUri']      
        ranking['metricValue'] = entry['chartEntryData']['rankingMetric']['value']
        ranking['metricType'] = entry['chartEntryData']['rankingMetric']['type']
        rankings.append(ranking.copy())

    return rankings

In [12]:
start_date = datetime.datetime(2025, 5, 15)

# Create empty pandas DataFrames to store the rankings and artists
chartType = "song"


rankings_chart_db = []
for country_code in all_counties:
    json_data, status_code = get_weekly_song_rankings('latest', country_code.lower(), chartTypes[chartType])
    if status_code != 200:
        print(f"Failed to get data for {country_code} with status code {status_code}")
        if status_code == 401:
            print("Token expired, please update the bearer token.")
            print(f"Stopped at {country_code}, {date.strftime(date_format)}")
            break
        continue
    earliest_date = json_data['displayChart']['chartMetadata']['dimensions']['earliestDate']
    date = start_date

    while date.strftime(date_format) >= earliest_date:
        date_str = date.strftime(date_format)
        country_lower = country_code.lower()
        json_data, status_code = get_weekly_song_rankings(date_str, country_lower, chartTypes[chartType])
        if status_code != 200:
            with open("failed.txt", "a") as f:
                f.write(f"{chartType},{date_str},{country_code}\n")
            if status_code == 401:
                print("Token expired, please update the bearer token.")
                print(f"Stopped at {country_code}, {date.strftime(date_format)}")
                break
            continue

        ranking = weekly_to_lines(json_data)
        rankings_chart_db.extend(ranking)

        # Move date to previous week
        date -= datetime.timedelta(weeks=1)


Token expired, please update the bearer token.
Stopped at NL, 2021-04-22
Failed to get data for NZ with status code 401
Token expired, please update the bearer token.
Stopped at NZ, 2021-04-22


In [13]:
# Save databases
chart_df = pd.DataFrame(rankings_chart_db)
chart_df.to_csv(os.path.join(database_dir, f'{chartType}.csv'), index=False)
print(len(chart_df))

3611903


In [14]:
fetched = []

with open("failed.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        split = line.split(",")
        chart = split[0].strip()
        date = split[1].strip()
        country = split[2].strip().lower()

        json_data = get_weekly_song_rankings(date, country, chartTypes[chart])
        rankings = weekly_to_lines(json_data)
        if json_data is None:
            print(f"Error at {date}, {country} due to error.")
            continue

        fetched.extend(rankings)

TypeError: tuple indices must be integers or slices, not str

In [ ]:
original_df = pd.read_csv(os.path.join(database_dir, f'{chartType}.csv'))
chart_df = pd.DataFrame(fetched[chartType])
chart_df = pd.concat([original_df, chart_df], ignore_index=True)
print(len(chart_df))
chart_df.to_csv(os.path.join(database_dir, f'{chartType}.csv'), index=False)
# Save to pickle as well
chart_df.to_pickle(os.path.join(database_dir, f'{chartType}.pkl'))